In [20]:
import pickle, gensim, logging
import pandas as pd
import numpy as np
from nltk.corpus import stopwords 
from gensim.models.keyedvectors import KeyedVectors

In [25]:
# Import the data 

data = pickle.load(open('onetsoccode.p', 'rb'))
jobtitles = pickle.load(open('lookuptable', 'rb'))

# Stop words 
stopWords = stopwords.words('english')

In [27]:
# Convert to DF

df = pd.DataFrame(data)
df2 = pd.DataFrame(jobtitles).T

data = pd.merge(df, 
                  df2, 
                  left_on = 'soc', 
                  right_on = 'soc', 
                  how = 'left'
                 )

In [46]:
# Function to preprocess the text 

def preprocessText(txt, stopwords): 
    """Use gensim simple preprocess and remove stopwords"""
    stopwords = stopwords
    txt = gensim.utils.simple_preprocess(txt)
    return  [word for word in txt if word not in stopwords]
    

In [31]:
# Preprocess

data['processed'] = data.apply(lambda x: preprocessText(x['text'], stopWords), axis = 1)

In [32]:
# import modules & set up logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [7]:
# Pretrained vectors from google
### https://code.google.com/archive/p/word2vec/

word_vectors = KeyedVectors.load_word2vec_format('/home/craig/Documents/googledata/GoogleNews-vectors-negative300.bin', binary=True)

2017-11-01 21:36:07,827 : INFO : loading projection weights from /home/craig/Documents/googledata/GoogleNews-vectors-negative300.bin
2017-11-01 21:37:31,568 : INFO : loaded (3000000, 300) matrix from /home/craig/Documents/googledata/GoogleNews-vectors-negative300.bin


In [39]:
# Function to create a numpy array of the model word_vec 

def average_word_embedding(data, model): 
    """Return the average word embedding"""
    out = []
    for word in data: 
        if word in model:
            out.append(model.word_vec(word))
    return np.average(np.array(out))

In [41]:
# Create the avg word vectors 
# -- This doesn't 'feel' right because it's going to use 
# -- all the words not just say a series of 300 

data['avg'] = data.apply(lambda x: average_word_embedding(x['processed'], word_vectors), axis = 1)

In [44]:
# Cosine Similarity -- Will return a matrix of the cosine similarity

from sklearn.metrics.pairwise import cosine_similarity

# cosine_similarity(data['avg'])
# -- seems like this should be a matrix not a single value 